In [1]:
import gdal2tiles
import glob
import numpy as np
import numpy.ma as ma
import os
from osgeo import gdal, gdalconst, osr
import rasterio
import rasterio.merge
from rasterio.windows import Window

%run _aws.ipynb
%run _constants.ipynb

In [2]:


def create_blank_tif(bbox_poly_ea, dst_dir):

    temp_bounds_ea = bbox_poly_ea.bounds
    xmin_ea = temp_bounds_ea[0]
    xmax_ea = temp_bounds_ea[2] 
    ymin_ea = temp_bounds_ea[1]
    ymax_ea = temp_bounds_ea[3]

    driver = gdal.GetDriverByName('GTiff')
    spatref = osr.SpatialReference()
    spatref.ImportFromEPSG(3857)
    wkt = spatref.ExportToWkt()

    blank_path = f'{dst_dir}/blank.tif'
    nbands = 1
    xres_ea = 10
    yres_ea = -10

    dtype = gdal.GDT_UInt16

    xsize = int(np.rint(np.abs((xmax_ea - xmin_ea)) / xres_ea))
    ysize = int(np.rint(np.abs((ymax_ea - ymin_ea) / yres_ea)))

    ds = driver.Create(blank_path, xsize, ysize, nbands, dtype, options=['COMPRESS=LZW', 'TILED=YES'])
    ds.SetProjection(wkt)
    ds.SetGeoTransform([xmin_ea, xres_ea, 0, ymax_ea, 0, yres_ea])
    ds.GetRasterBand(1).Fill(NODATA_UINT16)
    ds.GetRasterBand(1).SetNoDataValue(NODATA_UINT16)
    ds.FlushCache()
    ds = None

    return blank_path

In [3]:


def create_composite(band, stack_path, dst_dir, method="median"):
    
    if not os.path.exists(stack_path):
        raise ValueError(f'{stack_path} does not exist')

    with rasterio.open(stack_path) as stack_src:
    
        width, height = stack_src.width, stack_src.height

        composite_path = f'{dst_dir}/{band}_composite.tif'
        if os.path.exists(composite_path):
            return composite_path
        
        meta = stack_src.meta.copy()
        meta.update(count=1)
        
        with rasterio.open(composite_path, "w", **meta) as composite_dst:
            for row in range(height):    
                chunk = stack_src.read(window=Window(0, row, width, 1), masked=True)
                centre = np.rint(ma.median(chunk, axis=0)).astype(np.uint16)
                centre_data = centre.data
                centre_data[centre.mask] = NODATA_UINT16   
                composite_dst.write(centre_data, window=Window(0, row, width, 1), indexes=1)

    return composite_path

In [4]:


def create_band_stack(band, tif_paths, dst_dir):
    
    # band_paths = glob.glob(f'/tmp/{dir_name}/*/{band}_masked.tif')
    with rasterio.open(tif_paths[0]) as meta_src:
        meta = meta_src.meta.copy()

    meta.update(count=len(tif_paths))

    stack_path = f'{dst_dir}/{band}_stack.tif'
    with rasterio.open(stack_path, 'w', **meta) as stack_dst:
        for id, layer in enumerate(tif_paths, start=1):
            with rasterio.open(layer) as lyr_src:
                stack_dst.write_band(id, lyr_src.read(1))

    return stack_path

In [5]:


def merge_tif_with_blank(tif_path, blank_path, band, bbox_poly_ea, dst_dir):

    with rasterio.open(blank_path) as blank_src:
        with rasterio.open(tif_path) as tif_src:
            
            # if the bounds are the same then just skip merging
            tbnds, dbnds = blank_src.bounds, tif_src.bounds
            if tbnds.left == dbnds.left and tbnds.bottom == dbnds.bottom and \
            tbnds.right == dbnds.right and tbnds.top == dbnds.top:
                return

            merged, transform_ = rasterio.merge.merge([tif_src, blank_src], bounds=bbox_poly_ea.bounds)
            merged = merged[0, :, :]

            merged_profile = blank_src.profile.copy()
            if band_name == "SCL":
                merged_profile["dtype"] = "uint8"
                merged_profile["nodata"] = NODATA_BYTE

    merged_path = f'{dst_dir}/{band}_merged.tif'
    with rasterio.open(merged_path, "w", **merged_profile) as new_src:
        new_src.write(merged, 1)

    return merged_path

In [ ]:


def save_tif_to_s3(task_uid, tif_path, subdir):

    file_name = tif_path.split('/')[-1]
    object_key = f'{task_uid}/{subdir}/{file_name}'
    
    print(f'uploading {tif_path} to s3://{S3_TASKS_BUCKET}/{object_key}')

    s3_utils.put_item(tif_path, S3_TASKS_BUCKET, object_key)

    
    
def save_photo_to_s3(task_uid, photo_path, subdir):

    file_name = photo_path.split('/')[-1]
    object_key = f'{task_uid}/{subdir}/{file_name}'

    print(f'uploading {photo_path} to s3://{S3_TASKS_BUCKET}/{object_key}')

    s3_utils.put_item(photo_path, S3_TASKS_BUCKET, object_key)
    
    
    
def save_tiles_dir_to_s3(task_uid, tiles_dir, subdir):

    print(f'saving {tiles_dir} to S3')

    for root, dirs, files in os.walk(tiles_dir):
        for file in files:
            file_path = os.path.join(root, file)
            subpath = file_path.replace(tiles_dir, '')
            object_key = f'{task_uid}/{subdir}/tiles/{subpath}'
            
            s3_utils.put_item(file_path, S3_TASKS_BUCKET, object_key)


In [ ]:
from osgeo import gdal, gdalconst



def create_vrt(band_paths, dst_path):

    vrt_options = gdal.BuildVRTOptions(separate=True)
    gdal.BuildVRT(dst_path, band_paths, options=vrt_options)


def create_tif(vrt_path, dst_path, isCog=False):

    _format = "COG" if isCog else "TIF"

    translate_options = gdal.TranslateOptions(
        format=_format, 
        noData=65535, # TODO: add constants
    )

    gdal.Translate(dst_path, vrt_path, options=translate_options)


def create_byte_vrt(vrt_path, dst_path):

    translate_options = gdal.TranslateOptions(
        format="VRT", 
        outputType=gdalconst.GDT_Byte, 
        scaleParams=[[0, 2000, 0, 255]],
        noData=255, # TODO: add constants
    )
    gdal.Translate(dst_path, vrt_path, options=translate_options)
 


In [ ]:


def create_rgb_cog(composites, dst_path):

    vrt_path = '/tmp/temp_rgb.vrt' # TODO: change this... no hard coding paths
    band_paths = [composites['B04'], composites['B03'], composites['B02']]
    create_vrt(band_paths, vrt_path)
    create_tif(vrt_path, dst_path, isCog=True)
    

def create_full_tif(composites, dst_path):

    vrt_path = '/tmp/temp_full.vrt'
    band_paths = [composites[band] for band in composites]
    create_vrt(band_paths, vrt_path)
    create_tif(vrt_path, dst_path, isCog=False)


def create_byte_rgb_vrt(composites, dst_path):

    band_paths = [composites['B04'], composites['B03'], composites['B02']]
    vrt_path = f'/tmp/temp_rgb_byte_.vrt'
    create_vrt(band_paths, vrt_path)
    create_byte_vrt(vrt_path, dst_path)


In [ ]:


def create_rgb_map_tiles(rgb_vrt, step):
    tiles_dir = f'/tmp/{step}/tiles/' # TODO: no hard coding paths
    print(f'generating tiles from {rgb_vrt}: {tiles_dir}')

    options = {
        'kml': True,
        'nb_processes': 4,
        'title': 'Smart Carte',
        'zoom': (2, 12),
    }

    gdal2tiles.generate_tiles(rgb_vrt, tiles_dir, **options)
    return tiles_dir
